In [ ]:
%load_ext autoreload
%autoreload 2

from krxns.net_construction import construct_reaction_network, extract_compounds
from krxns.config import filepaths
from krxns.utils import str2int
from krxns.cheminfo import draw_reaction
from krxns.networks import SuperMultiDiGraph
import json
from IPython.display import SVG

In [25]:
# Load unpaired cofactors
with open(filepaths['cofactors'] / 'expanded_unpaired_cofactors.json', 'r') as f:
    unpaired_cofactors = json.load(f)

# Load known reaction data
with open(filepaths['data'] / 'sprhea_240310_v3_mapped.json', 'r') as f:
    krs = json.load(f)

# Load op connected reactions
with open(filepaths['connected_reactions'] / 'sprhea_240310_v3_mapped_operator.json', 'r') as f:
    op_cxns = str2int(json.load(f))

# Load sim connected reactions
with open(filepaths['connected_reactions'] / 'sprhea_240310_v3_mapped_similarity.json', 'r') as f:
    sim_cxn = str2int(json.load(f))

with open(filepaths['connected_reactions'] / 'sprhea_240310_v3_mapped_side_counts.json', 'r') as f:
    side_counts = str2int(json.load(f))

# Load coreactant whitelist
with open(filepaths['cofactors'] / 'pickaxe_whitelist.json', 'r') as f:
    coreactant_whitelist = json.load(f)


# Get known compounds
kcs, smi2id = extract_compounds(krs)

In [26]:
G = SuperMultiDiGraph()
edges, nodes = construct_reaction_network(
    operator_connections=op_cxns,
    similarity_connections=sim_cxn,
    side_counts=side_counts,
    reactions=krs,
    unpaired_cofactors=unpaired_cofactors,
    connect_nontrivial=False,
    # atom_lb=0.6,
    coreactant_whitelist=coreactant_whitelist
)
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [69]:
def show_path(source, target, topk = 1):
    node_path, edge_path = G.shortest_path(source, target)
    for i, step in enumerate(edge_path):
        print(f"STEP #{i+1}")
        for edge in step[:topk]:
            print([(k, v) for k, v in edge.items() if k != 'smarts'])
            display(SVG(draw_reaction(edge['smarts'])))

In [ ]:
i = G.get_nodes_by_prop('name', 'pyruvate')[0]
j  = G.get_nodes_by_prop('name', 'succinate')[0]
show_path(i, j)

In [ ]:
show_path(j, i)

In [ ]:
i = G.get_nodes_by_prop('name', 'butanoate')[0]
j  = G.get_nodes_by_prop('name', 'butanoyl-CoA')[0]
show_path(i, j)

In [ ]:
i = G.get_nodes_by_prop('name', 'pyruvate')[0]
j  = G.get_nodes_by_prop('name', 'acetone')[0]
show_path(i, j)

In [ ]:
show_path(j, i)

In [ ]:
# i = G.get_nodes_by_prop('name', 'pyruvate')[0]
# j  = G.get_nodes_by_prop('name', 'L-lysine')[0]
# show_path(i, j)

In [ ]:
sorted([node['name'] for id, node in G.nodes(data=True)], key=lambda x : len(x))

In [ ]:
# smi = standardize_smiles('CC(C)(CO)[C@@H](O)C(=O)NCCC(=O)[O-]') # CCC(C)(C)C(=O)SCCC(=O)OC
# ids = G.get_nodes_by_prop('smiles', smi)
# ids

In [ ]:
# n = 2
# id = ids[0]
# n_hop_neighbors = nx.ego_graph(G, id, radius=n)
# for i in n_hop_neighbors:
#     print(G.nodes[i]['name'])
#     display(SVG(draw_molecule(G.nodes[i]['smiles'])))

In [ ]:
# for i in n_hop_neighbors:
#     print(f"PATH FROM {G.nodes[i]['name']}")
#     show_path(i, id)